# Attempt At a Wordle Solving Bot

set up necessary imports

In [1]:
!pip install nltk

In [1]:
import random
import nltk
from nltk.corpus import words
from collections import Counter
import math

Load list of words from the .txt file, filter them accordingly into new file containing only common words from nltk for ease of use for this test.

In [2]:
# Download NLTK data
#nltk.download('words')

# Load a list of valid 5-letter words from a file
with open('wordle-word-list.txt', 'r') as file:
    word_list = file.read().splitlines()

# Get the list of common English words from NLTK
nltk_words = set(words.words())

# Filter the word list to include only common words
filtered_word_list = [word for word in word_list if word in nltk_words]

# Save the filtered word list to a new file
with open('filtered-wordle-word-list.txt', 'w') as file:
    for word in filtered_word_list:
        file.write(f"{word}\n")

print(f"Filtered word list saved. Original list had {len(word_list)} words, filtered list has {len(filtered_word_list)} words.")

Filtered word list saved. Original list had 14855 words, filtered list has 6111 words.


## Feedback

Feedback function for displaying the progress of the game. Currently this will be used for both human and bot based feedback reports. 
Simple GUI based on the worlde gameboard to allow for better visibility.

In [3]:
def get_feedback(guess, solution):
    feedback = [''] * 5
    solution_chars = list(solution)
    guess_chars = list(guess)
    
    # First pass: find correct letters (🟩)
    for i in range(5):
        if guess_chars[i] == solution_chars[i]:
            feedback[i] = '🟩'
            solution_chars[i] = None  # Remove matched character to avoid duplicates
    
    # Second pass: find misplaced letters (🟨)
    for i in range(5):
        if feedback[i] == '':
            if guess_chars[i] in solution_chars:
                feedback[i] = '🟨'
                solution_chars[solution_chars.index(guess_chars[i])] = None
            else:
                feedback[i] = '⬛'  # Incorrect letters (⬛)
    
    return ''.join(feedback)

## Play the wordle game, USER BASED, !!NOT!! BOT

In [ ]:
def play_wordle(solution):
    guesses = []
    while len(guesses) < 6:
        guess = input("Enter your guess: ").lower()
        if len(guess) != 5:
            print("Guess must be exactly 5 letters long.")
            continue
        feedback = get_feedback(guess, solution)
        print(f"Feedback: {feedback}")
        guesses.append((guess, feedback))
        if guess == solution:
            print("Congratulations! You guessed the word!")
            break
    else:
        print(f"Sorry, you did not guess the word. The correct word was: {solution}")
    return guesses

# Play the game
solution = random.choice(filtered_word_list)
guesses = play_wordle(solution)

## Bot Guess Logic 

Currently undefined and using a brute force method in which random guesses are given and no learning is used.

## Brute Force ##

In [4]:
def bot_brute_guess(word_list, feedback_history):
    return random.choice(word_list)

## Simple Filter

In [5]:
def bot_guess_simple(word_list, feedback_history):
    # Filter words based on previous feedback
    possible_words = word_list
    for guess, feedback in feedback_history:
        new_possible_words = []
        for word in possible_words:
            if get_feedback(guess, word) == feedback:
                new_possible_words.append(word)
        possible_words = new_possible_words
    return random.choice(possible_words) if possible_words else random.choice(word_list)

## Frequency Based

In [6]:
def bot_guess_frequency(word_list, feedback_history):
    # Filter words based on previous feedback
    possible_words = word_list
    for guess, feedback in feedback_history:
        new_possible_words = []
        for word in possible_words:
            if get_feedback(guess, word) == feedback:
                new_possible_words.append(word)
        possible_words = new_possible_words
    
    # Calculate letter frequencies
    letter_frequencies = Counter(''.join(possible_words))
    
    # Score words based on letter frequencies
    def score_word(word):
        return sum(letter_frequencies[letter] for letter in set(word))
    
    best_guess = max(possible_words, key=score_word)
    return best_guess if possible_words else random.choice(word_list)

## Custom Algorithm Based off My Approach

In [23]:
def bot_custom_guess(word_list, feedback_history):
    # Define the initial guess with multiple vowels
    initial_vowel_guess = "audio"  # Can be replaced with any word with multiple vowels

    if not feedback_history:
        return initial_vowel_guess

    # Track letters based on feedback
    correct_positions = {}
    misplaced_letters = {}
    incorrect_letters = set()

    for guess, feedback in feedback_history:
        for i, (char, fb) in enumerate(zip(guess, feedback)):
            if fb == '🟩':
                correct_positions[i] = char
            elif fb == '🟨':
                if char not in misplaced_letters:
                    misplaced_letters[char] = []
                misplaced_letters[char].append(i)
            else:
                incorrect_letters.add(char)

    # Filter possible words
    possible_words = word_list
    for guess, feedback in feedback_history:
        new_possible_words = []
        for word in possible_words:
            match = True
            for i, (char, fb) in enumerate(zip(guess, feedback)):
                if fb == '🟩' and word[i] != char:
                    match = False
                elif fb == '🟨' and (word[i] == char or char not in word):
                    match = False
                elif fb == '⬛' and char in word:
                    match = False
            if match:
                new_possible_words.append(word)
        possible_words = new_possible_words

    # Further filter based on correct positions and misplaced letters
    new_possible_words = []
    for word in possible_words:
        match = True
        for i, char in correct_positions.items():
            if word[i] != char:
                match = False
                break
        for char, indices in misplaced_letters.items():
            if char not in word or any(word[i] == char for i in indices):
                match = False
                break
        if match and all(char not in incorrect_letters for char in word):
            new_possible_words.append(word)
    possible_words = new_possible_words
    
    # If no word is found, return a random word from possible words
    if possible_words:
        return random.choice(possible_words)
    else:
        # As a fallback, return a random word from the original list if filtering is too strict
        return random.choice(word_list)

## Wordle Bot Rules and Function. 

In [13]:
def play_wordle_bot(solution, word_list, bot_guess_function):
    guesses = []
    feedback_history = []
    while len(guesses) < 6:
        guess = bot_guess_function(word_list, feedback_history)
        print(f"Bot guess: {guess}")  # Debugging print
        if len(guess) != 5:
            print("Invalid guess length, skipping...")  # Debugging print
            continue  # Ensure the bot guess is exactly 5 letters long
        feedback = get_feedback(guess, solution)
        print(f"Feedback: {feedback}")  # Debugging print
        feedback_history.append((guess, feedback))
        guesses.append((guess, feedback))
        if guess == solution:
            print(f"Bot guessed the word! Correct word was: {solution}")
            break
    else:
        print(f"Bot did not guess the word. The correct word was: {solution}")
    return guesses

## Execute Bots and Comparison

In [27]:
# Define the strategies
strategies = {
    'brute': bot_brute_guess,
    'simple': bot_guess_simple,
    'frequency': bot_guess_frequency,
    'custom': bot_custom_guess
}

# Play the game with each strategy and record the results
results = {}
for strategy_name, strategy_function in strategies.items():
    print(f"\nRunning strategy: {strategy_name}")
    solution = random.choice(filtered_word_list)
    guesses = play_wordle_bot(solution, filtered_word_list, strategy_function)
    results[strategy_name] = len(guesses)

# Print the results
for strategy_name, num_guesses in results.items():
    print(f"Strategy {strategy_name} solved the puzzle in {num_guesses} guesses")


Running strategy: brute
Bot guess: frosh
Feedback: ⬛⬛⬛⬛🟩
Bot guess: phage
Feedback: ⬛🟨🟨⬛🟨
Bot guess: manso
Feedback: ⬛🟩🟩⬛⬛
Bot guess: strew
Feedback: ⬛⬛⬛🟩⬛
Bot guess: maize
Feedback: ⬛🟩⬛⬛🟨
Bot guess: kokra
Feedback: 🟩⬛⬛⬛🟨
Bot did not guess the word. The correct word was: kaneh

Running strategy: simple
Bot guess: mauve
Feedback: ⬛⬛⬛⬛⬛
Bot guess: sissy
Feedback: ⬛🟩⬛⬛⬛
Bot guess: dildo
Feedback: ⬛🟩🟩⬛⬛
Bot guess: hilch
Feedback: ⬛🟩🟩🟩🟩
Bot guess: filch
Feedback: ⬛🟩🟩🟩🟩
Bot guess: pilch
Feedback: 🟩🟩🟩🟩🟩
Bot guessed the word! Correct word was: pilch

Running strategy: frequency
Bot guess: arose
Feedback: ⬛⬛⬛🟨🟩
Bot guess: istle
Feedback: ⬛🟨⬛⬛🟩
Bot guess: ensue
Feedback: 🟨🟨🟨⬛🟩
Bot guess: scene
Feedback: 🟩🟩🟩🟩🟩
Bot guessed the word! Correct word was: scene

Running strategy: custom
Bot guess: audio
Feedback: 🟨⬛🟨⬛⬛
Bot guess: bardy
Feedback: ⬛🟨⬛🟩🟩
Bot guess: shady
Feedback: ⬛⬛🟩🟩🟩
Bot guess: leady
Feedback: 🟩🟩🟩🟩🟩
Bot guessed the word! Correct word was: leady
Strategy brute solved the puzzle in 6 gu